In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8082
WEBHOOK_BASE = ""
ADMIN_URL = "http://carol-agent:8081"
API_KEY = "carol_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)


IPython autoawait is `on`, and set to use `asyncio`


In [2]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

# connect 3 agents

## 2. connect to DM

In [3]:

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)

Subscribing too: connections


In [4]:
#Paste in invitation from Alice agent
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '51faa42a-8769-450c-815e-3d1b022134a2', 'recipientKeys': ['6uGSV3tKN3kbF3ZC5GmuxqHrASfsSKBe4Kxu9JzTdnYS'], 'serviceEndpoint': 'http://192.168.65.3:8020', 'label': 'Alice'}

In [5]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invitation)
# Print out accepted Invite and Alice's connection ID
print("Connection", response)
alice_id = response["connection_id"]


Connection Handler Called
Connection 85bc4fa6-f17c-47ab-a577-ec33f93afd6d in State invitation
Connection Handler Called
Connection 85bc4fa6-f17c-47ab-a577-ec33f93afd6d in State request
Connection {'state': 'request', 'invitation_key': '6uGSV3tKN3kbF3ZC5GmuxqHrASfsSKBe4Kxu9JzTdnYS', 'updated_at': '2023-01-03 12:19:07.186468Z', 'accept': 'manual', 'their_label': 'Alice', 'request_id': '6e18face-4202-47ca-9ce5-62f7c6165976', 'rfc23_state': 'request-sent', 'invitation_mode': 'once', 'routing_state': 'none', 'their_role': 'inviter', 'my_did': 'MBSF9UmbuFnAftoJx2pw1j', 'connection_id': '85bc4fa6-f17c-47ab-a577-ec33f93afd6d', 'created_at': '2023-01-03 12:19:07.126280Z'}
Connection Handler Called
Connection 85bc4fa6-f17c-47ab-a577-ec33f93afd6d in State response
Connection Handler Called
Connection 85bc4fa6-f17c-47ab-a577-ec33f93afd6d in State active


### 3. switch to DM

### 4.

In [6]:
# Print connection list
connection = await agent_controller.connections.get_connection(alice_id)
print("Alice AGENT CONNECTION")
print(connection)
print("State:", connection["state"])
dm_connection = connection["connection_id"]
print(dm_connection)

Alice AGENT CONNECTION
{'state': 'active', 'invitation_key': '6uGSV3tKN3kbF3ZC5GmuxqHrASfsSKBe4Kxu9JzTdnYS', 'updated_at': '2023-01-03 12:19:13.347592Z', 'accept': 'manual', 'their_label': 'Alice', 'request_id': '6e18face-4202-47ca-9ce5-62f7c6165976', 'rfc23_state': 'completed', 'their_did': 'WVwvP9Tn18f1eZ8pi7DfTL', 'invitation_mode': 'once', 'routing_state': 'none', 'their_role': 'inviter', 'my_did': 'MBSF9UmbuFnAftoJx2pw1j', 'connection_id': '85bc4fa6-f17c-47ab-a577-ec33f93afd6d', 'created_at': '2023-01-03 12:19:07.126280Z'}
State: active


In [7]:
# dm_connection = 'e75d05c2-aab0-47d1-b2e5-43415e076121'
print(dm_connection)

85bc4fa6-f17c-47ab-a577-ec33f93afd6d


In [8]:
response = await agent_controller.connections.get_connections()
print(response)

{'results': [{'state': 'active', 'invitation_key': '6uGSV3tKN3kbF3ZC5GmuxqHrASfsSKBe4Kxu9JzTdnYS', 'updated_at': '2023-01-03 12:19:13.347592Z', 'accept': 'manual', 'their_label': 'Alice', 'request_id': '6e18face-4202-47ca-9ce5-62f7c6165976', 'rfc23_state': 'completed', 'their_did': 'WVwvP9Tn18f1eZ8pi7DfTL', 'invitation_mode': 'once', 'routing_state': 'none', 'their_role': 'inviter', 'my_did': 'MBSF9UmbuFnAftoJx2pw1j', 'connection_id': '85bc4fa6-f17c-47ab-a577-ec33f93afd6d', 'created_at': '2023-01-03 12:19:07.126280Z'}]}


## 5. connect to MUSIC

In [9]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)

Connection Handler Called
Connection 37f5ab65-a1a6-439d-9933-f79b98b15ef9 in State invitation
Connection ID 37f5ab65-a1a6-439d-9933-f79b98b15ef9
Invitation
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '41081439-7db1-4060-ace9-4a2a618a941b', 'recipientKeys': ['CKwsvo1gh2VjE6BfChqfi171X6AzMee11pRqbnCQQtfe'], 'label': 'Carol', 'serviceEndpoint': 'http://192.168.65.3:8080'}
Connection Handler Called
Connection 37f5ab65-a1a6-439d-9933-f79b98b15ef9 in State request


### 6. switch to MUSIC

### 7.

In [10]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

Connection Handler Called
Connection 37f5ab65-a1a6-439d-9933-f79b98b15ef9 in State response
ACCEPT REQUEST
{'state': 'response', 'invitation_key': 'CKwsvo1gh2VjE6BfChqfi171X6AzMee11pRqbnCQQtfe', 'updated_at': '2023-01-03 12:19:53.169973Z', 'accept': 'manual', 'their_label': 'Bob', 'rfc23_state': 'response-sent', 'their_did': 'AfY42mQdAawA3CMpusYpLN', 'invitation_mode': 'once', 'routing_state': 'none', 'their_role': 'invitee', 'my_did': '9YrzDVXwE1e9UNk7rgfCET', 'connection_id': '37f5ab65-a1a6-439d-9933-f79b98b15ef9', 'created_at': '2023-01-03 12:19:31.565180Z'}
state response


In [11]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

Trust Ping {'thread_id': 'e29b2e51-fb96-4afe-9f8f-09c20fdfe73c'}
Connection Handler Called
Connection 37f5ab65-a1a6-439d-9933-f79b98b15ef9 in State active


In [12]:
connection = await agent_controller.connections.get_connection(connection_id)
print(connection)
print("Is Active?", connection["state"])
music_connection = connection["connection_id"]
print(music_connection)

{'state': 'active', 'invitation_key': 'CKwsvo1gh2VjE6BfChqfi171X6AzMee11pRqbnCQQtfe', 'updated_at': '2023-01-03 12:19:54.472003Z', 'accept': 'manual', 'their_label': 'Bob', 'rfc23_state': 'completed', 'their_did': 'AfY42mQdAawA3CMpusYpLN', 'invitation_mode': 'once', 'routing_state': 'none', 'their_role': 'invitee', 'my_did': '9YrzDVXwE1e9UNk7rgfCET', 'connection_id': '37f5ab65-a1a6-439d-9933-f79b98b15ef9', 'created_at': '2023-01-03 12:19:31.565180Z'}
Is Active? active
37f5ab65-a1a6-439d-9933-f79b98b15ef9


In [13]:
# music_connection = 'b616d290-fb08-4a09-9ab4-314fe92f0f25'

In [14]:
response = await agent_controller.connections.get_connections()
print(response)

{'results': [{'state': 'active', 'invitation_key': '6uGSV3tKN3kbF3ZC5GmuxqHrASfsSKBe4Kxu9JzTdnYS', 'updated_at': '2023-01-03 12:19:13.347592Z', 'accept': 'manual', 'their_label': 'Alice', 'request_id': '6e18face-4202-47ca-9ce5-62f7c6165976', 'rfc23_state': 'completed', 'their_did': 'WVwvP9Tn18f1eZ8pi7DfTL', 'invitation_mode': 'once', 'routing_state': 'none', 'their_role': 'inviter', 'my_did': 'MBSF9UmbuFnAftoJx2pw1j', 'connection_id': '85bc4fa6-f17c-47ab-a577-ec33f93afd6d', 'created_at': '2023-01-03 12:19:07.126280Z'}, {'state': 'active', 'invitation_key': 'CKwsvo1gh2VjE6BfChqfi171X6AzMee11pRqbnCQQtfe', 'updated_at': '2023-01-03 12:19:54.472003Z', 'accept': 'manual', 'their_label': 'Bob', 'rfc23_state': 'completed', 'their_did': 'AfY42mQdAawA3CMpusYpLN', 'invitation_mode': 'once', 'routing_state': 'none', 'their_role': 'invitee', 'my_did': '9YrzDVXwE1e9UNk7rgfCET', 'connection_id': '37f5ab65-a1a6-439d-9933-f79b98b15ef9', 'created_at': '2023-01-03 12:19:31.565180Z'}]}


In [15]:
response = await agent_controller.connections.get_connection(music_connection)
print(response)

{'state': 'active', 'invitation_key': 'CKwsvo1gh2VjE6BfChqfi171X6AzMee11pRqbnCQQtfe', 'updated_at': '2023-01-03 12:19:54.472003Z', 'accept': 'manual', 'their_label': 'Bob', 'rfc23_state': 'completed', 'their_did': 'AfY42mQdAawA3CMpusYpLN', 'invitation_mode': 'once', 'routing_state': 'none', 'their_role': 'invitee', 'my_did': '9YrzDVXwE1e9UNk7rgfCET', 'connection_id': '37f5ab65-a1a6-439d-9933-f79b98b15ef9', 'created_at': '2023-01-03 12:19:31.565180Z'}


In [81]:
# response = await agent_controller.connections.remove_connection('c5f441dd-41d8-4bdb-a041-0a5566e4a450')

### 8. switch to MUSIC

# exchange DID with DM

## 3.

In [16]:
my_did = await agent_controller.wallet.get_public_did()
my_did = my_did['result']['did']
print("my DID", my_did)
dm_did = 'None'

my DID X8HWQruWXrv8s6BBLXm53B


In [17]:
def messages_handler(payload):
    global dm_did
    connection_id = payload["connection_id"]
    print("Handle message", payload, connection_id)
    dm_did = payload['content']
    print('DM DID', dm_did)


message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([message_listener], defaults=True)

Subscribing too: basicmessages


In [18]:
# await agent_controller.messaging.trust_ping(dm_connection, "hello, world!")

In [19]:
response = await agent_controller.messaging.send_message(dm_connection, 'sp_'+my_did)
print(response)

{}
Handle message {'connection_id': '85bc4fa6-f17c-47ab-a577-ec33f93afd6d', 'message_id': '114a552b-796e-44c6-823e-eb15e5d21606', 'content': 'PQRXDxdGqQGSZ8z69p4xZP', 'state': 'received'} 85bc4fa6-f17c-47ab-a577-ec33f93afd6d
DM DID PQRXDxdGqQGSZ8z69p4xZP


In [20]:
print(dm_did)

PQRXDxdGqQGSZ8z69p4xZP


# Verify MUSIC

## 1. request proof from MUSIC (prove that MUSIC belongs to DM) 

In [21]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def proof_handler(payload):
    print("Handle present proof")
    role = payload["role"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print(f"Role {role}, Exchange {pres_ex_id} in state {state}")

proof_listener = {
    "topic": "present_proof",
    "handler": proof_handler
}
agent_controller.register_listeners([proof_listener], defaults=True)


Subscribing too: present_proof


## 2. switch to MUSIC

## 3.

In [22]:
response = await agent_controller.wallet.get_public_did()
print(response)
issuer_did = response["result"]["did"]

print("Request proof from MUSIC")
#Set some variables

revocation = False
SELF_ATTESTED = True
exchange_tracing = False

#Enable this to ask for attributes to identity a user
req_attrs = [
    {"name": "author", "restrictions": [{"issuer_did": dm_did}]},
    {"name": "origin", "restrictions": [{"issuer_did": dm_did}]},
    {"name": "owner", "restrictions": [{"issuer_did": dm_did}]},
    {"name": "time", "restrictions": [{"issuer_did": dm_did}]},
]

if revocation:
    req_attrs.append(
        {
            "name": "owner",
            "restrictions": [{"issuer_did": agent.did}],
            "non_revoked": {"to": int(time.time() - 1)},
        },
    )



#Set predicates for Zero Knowledge Proofs
req_preds = []

indy_proof_request = {
    "name": "Proof of Ownership of DM",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {
        f"0_{req_pred['name']}_GE_uuid":
        req_pred for req_pred in req_preds
    },
}

if revocation:
    indy_proof_request["non_revoked"] = {"to": int(time.time())}

#proof_request = indy_proof_request
exchange_tracing_id = exchange_tracing
proof_request_web_request = {
    "connection_id": music_connection,
    "proof_request": indy_proof_request,
    "trace": exchange_tracing,
}

{'result': {'did': 'X8HWQruWXrv8s6BBLXm53B', 'verkey': 'HRL7SCYGvMzoj6QJJ7MymnC3WZ7fXDrpvwV56sPT3zTr', 'posture': 'public'}}
Request proof from MUSIC


In [23]:
response = await agent_controller.proofs.send_request(proof_request_web_request)
print(response)
presentation_exchange_id = response['presentation_exchange_id']
print("\n")
print(presentation_exchange_id)

Handle present proof
Role verifier, Exchange b2ed21f5-28a1-4a03-ab23-0db76f988f94 in state request_sent
{'state': 'request_sent', 'trace': False, 'role': 'verifier', 'updated_at': '2023-01-03 12:28:55.178266Z', 'initiator': 'self', 'presentation_request_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation', '@id': 'e5c0ec43-084a-4671-b255-92484d67c147', 'request_presentations~attach': [{'@id': 'libindy-request-presentation-0', 'mime-type': 'application/json', 'data': {'base64': 'eyJuYW1lIjogIlByb29mIG9mIE93bmVyc2hpcCBvZiBETSIsICJ2ZXJzaW9uIjogIjEuMCIsICJyZXF1ZXN0ZWRfYXR0cmlidXRlcyI6IHsiMF9hdXRob3JfdXVpZCI6IHsibmFtZSI6ICJhdXRob3IiLCAicmVzdHJpY3Rpb25zIjogW3siaXNzdWVyX2RpZCI6ICJQUVJYRHhkR3FRR1NaOHo2OXA0eFpQIn1dfSwgIjBfb3JpZ2luX3V1aWQiOiB7Im5hbWUiOiAib3JpZ2luIiwgInJlc3RyaWN0aW9ucyI6IFt7Imlzc3Vlcl9kaWQiOiAiUFFSWER4ZEdxUUdTWjh6NjlwNHhaUCJ9XX0sICIwX293bmVyX3V1aWQiOiB7Im5hbWUiOiAib3duZXIiLCAicmVzdHJpY3Rpb25zIjogW3siaXNzdWVyX2RpZCI6ICJQUVJYRHhkR3FRR1NaOHo2

## 4. switch to MUSIC

## 5.

In [24]:
verify = await agent_controller.proofs.verify_presentation(presentation_exchange_id)

Handle present proof
Role verifier, Exchange b2ed21f5-28a1-4a03-ab23-0db76f988f94 in state verified


In [25]:
print(verify['state'])
music_belongs_to_dm = verify['state'] == 'verified'
print(music_belongs_to_dm)

verified
True


# transfer possession

## 1. request new VC from DM

In [26]:
agent_controller.remove_all_listeners("basicmessages")

In [ ]:
def request_handler(payload):
    connection_id = payload["connection_id"]
    print("Handle request", payload, connection_id)


request_listener = {
    "handler": request_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([request_listener], defaults=True)

## 2. switch to DM

## 3.

In [29]:
response = await agent_controller.messaging.send_message(dm_connection, str(music_belongs_to_dm))
print(response)

{}
Handle request {'connection_id': '85bc4fa6-f17c-47ab-a577-ec33f93afd6d', 'message_id': '430f501f-198b-4328-b6fa-22f23d388ba1', 'content': 'revoke: True', 'state': 'received'} 85bc4fa6-f17c-47ab-a577-ec33f93afd6d


## 4. switch to DM

## 9. 

In [33]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())
def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Attributes: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)

Subscribing too: issue_credential


## 10. switch to MUSIC

## 11.

In [34]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")


Credential exchange d989c85f-4ad5-4516-b736-dca8fa1a7850, role: holder, state: offer_received
Being offered: [{'name': 'possession', 'value': ''}, {'name': 'time', 'value': '01-03-2023, 12:35:32'}]
Handle Credentials
Credential exchange 305c9c8e-70d0-449f-8dcf-fa419b4c8e9c, role: holder, state: offer_received
Attributes: [{'name': 'possession', 'value': ''}, {'name': 'time', 'value': '01-03-2023, 12:35:32'}]


In [35]:
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Handle Credentials
Credential exchange d989c85f-4ad5-4516-b736-dca8fa1a7850, role: holder, state: request_sent
Attributes: [{'name': 'possession', 'value': ''}, {'name': 'time', 'value': '01-03-2023, 12:35:32'}]
Credential exchange d989c85f-4ad5-4516-b736-dca8fa1a7850, role: holder, state: request_sent
Handle Credentials
Credential exchange d989c85f-4ad5-4516-b736-dca8fa1a7850, role: holder, state: credential_received
Attributes: [{'name': 'possession', 'value': ''}, {'name': 'time', 'value': '01-03-2023, 12:35:32'}]


In [36]:
response = await agent_controller.issuer.store_credential(cred_ex_id, "Credentia of Possesion")
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Handle Credentials
Credential exchange d989c85f-4ad5-4516-b736-dca8fa1a7850, role: holder, state: credential_acked
Attributes: [{'name': 'possession', 'value': ''}, {'name': 'time', 'value': '01-03-2023, 12:35:32'}]
Credential exchange d989c85f-4ad5-4516-b736-dca8fa1a7850, role: holder, state: credential_acked
Handle present proof
Role prover, Exchange 89655621-d1b5-4a39-a2b2-f109b81a0780 in state request_received


## 12. switch to MUSIC

## 13. present proof of possesion to MUSIC

In [40]:
response = await agent_controller.proofs.get_records()
print((response['results'][0]))

print('\n')

state = response['results'][0]["state"]
presentation_exchange_id = response['results'][0]['presentation_exchange_id']
presentation_request = response['results'][0]['presentation_request']

print('Presentation Exchange ID')
print(response['results'][0]['presentation_exchange_id'])
print('Presentation Request Object')
print(response['results'][0]['presentation_request'])
print('Requested Attributes')
print(response['results'][0]['presentation_request']['requested_attributes'])
requested_attribs = response['results'][0]['presentation_request']['requested_attributes']
print('Requested Predicates')
print(response['results'][0]['presentation_request']['requested_predicates'])
requested_predicates = response['results'][0]['presentation_request']['requested_predicates']

{'state': 'request_received', 'trace': False, 'role': 'prover', 'updated_at': '2023-01-03 12:36:33.518703Z', 'initiator': 'external', 'presentation_request_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation', '@id': 'e8791044-eccd-4f6c-b1fa-ce46fb7955d7', 'request_presentations~attach': [{'@id': 'libindy-request-presentation-0', 'mime-type': 'application/json', 'data': {'base64': 'eyJuYW1lIjogIlByb29mIG9mIE93bmVyc2hpcCIsICJ2ZXJzaW9uIjogIjEuMCIsICJyZXF1ZXN0ZWRfYXR0cmlidXRlcyI6IHsiMF9wb3NzZXNzaW9uX3V1aWQiOiB7Im5hbWUiOiAicG9zc2Vzc2lvbiIsICJyZXN0cmljdGlvbnMiOiBbeyJpc3N1ZXJfZGlkIjogIlBRUlhEeGRHcVFHU1o4ejY5cDR4WlAifV19LCAiMF90aW1lX3V1aWQiOiB7Im5hbWUiOiAidGltZSIsICJyZXN0cmljdGlvbnMiOiBbeyJpc3N1ZXJfZGlkIjogIlBRUlhEeGRHcVFHU1o4ejY5cDR4WlAifV19fSwgInJlcXVlc3RlZF9wcmVkaWNhdGVzIjoge30sICJub25jZSI6ICIxMTcyNjkyMjE4ODUyOTIyNjAzNTc5MDE0In0='}}]}, 'thread_id': 'e8791044-eccd-4f6c-b1fa-ce46fb7955d7', 'presentation_exchange_id': '89655621-d1b5-4a39-a2b2-f109b81a0

In [41]:
if state == "request_received":
    print(
    "Received Request -> Query for credentials in the wallet that satisfy the proof request")

# include self-attested attributes (not included in credentials)
credentials_by_reft = {}
revealed = {}
self_attested = {}
predicates = {}

# select credentials to provide for the proof
credentials = await agent_controller.proofs.get_presentation_credentials(presentation_exchange_id)
print(credentials)

if credentials:
    for row in sorted(
        credentials,
        key=lambda c: dict(c["cred_info"]["attrs"]),
        reverse=True,
    ):
        for referent in row["presentation_referents"]:
            if referent not in credentials_by_reft:
                credentials_by_reft[referent] = row

for referent in presentation_request["requested_attributes"]:
    if referent in credentials_by_reft:
        revealed[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ],
            "revealed": True,
        }


for referent in presentation_request["requested_predicates"]:
    if referent in credentials_by_reft:
        predicates[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ]
        }

print("\nGenerate the proof")
proof = {
    "requested_predicates": predicates,
    "requested_attributes": revealed,
    "self_attested_attributes": self_attested,
}
print(proof)
print("\nXXX")
print(predicates)
print(revealed)
print(self_attested)

Received Request -> Query for credentials in the wallet that satisfy the proof request
[{'cred_info': {'referent': 'Credentia of Possesion', 'attrs': {'possession': '', 'time': '01-03-2023, 12:35:32'}, 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:possession schema:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:14:default', 'rev_reg_id': None, 'cred_rev_id': None}, 'interval': None, 'presentation_referents': ['0_possession_uuid', '0_time_uuid']}]

Generate the proof
{'requested_predicates': {}, 'requested_attributes': {'0_possession_uuid': {'cred_id': 'Credentia of Possesion', 'revealed': True}, '0_time_uuid': {'cred_id': 'Credentia of Possesion', 'revealed': True}}, 'self_attested_attributes': {}}

XXX
{}
{'0_possession_uuid': {'cred_id': 'Credentia of Possesion', 'revealed': True}, '0_time_uuid': {'cred_id': 'Credentia of Possesion', 'revealed': True}}
{}


In [42]:
response = await agent_controller.proofs.send_presentation(presentation_exchange_id, proof)
print(response)

{'state': 'presentation_sent', 'trace': False, 'role': 'prover', 'updated_at': '2023-01-03 12:37:45.635257Z', 'initiator': 'external', 'presentation_request_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation', '@id': 'e8791044-eccd-4f6c-b1fa-ce46fb7955d7', 'request_presentations~attach': [{'@id': 'libindy-request-presentation-0', 'mime-type': 'application/json', 'data': {'base64': 'eyJuYW1lIjogIlByb29mIG9mIE93bmVyc2hpcCIsICJ2ZXJzaW9uIjogIjEuMCIsICJyZXF1ZXN0ZWRfYXR0cmlidXRlcyI6IHsiMF9wb3NzZXNzaW9uX3V1aWQiOiB7Im5hbWUiOiAicG9zc2Vzc2lvbiIsICJyZXN0cmljdGlvbnMiOiBbeyJpc3N1ZXJfZGlkIjogIlBRUlhEeGRHcVFHU1o4ejY5cDR4WlAifV19LCAiMF90aW1lX3V1aWQiOiB7Im5hbWUiOiAidGltZSIsICJyZXN0cmljdGlvbnMiOiBbeyJpc3N1ZXJfZGlkIjogIlBRUlhEeGRHcVFHU1o4ejY5cDR4WlAifV19fSwgInJlcXVlc3RlZF9wcmVkaWNhdGVzIjoge30sICJub25jZSI6ICIxMTcyNjkyMjE4ODUyOTIyNjAzNTc5MDE0In0='}}]}, 'thread_id': 'e8791044-eccd-4f6c-b1fa-ce46fb7955d7', 'presentation_exchange_id': '89655621-d1b5-4a39-a2b2-f109b81a

## 14. switch to MUSIC

In [43]:
response = await agent_controller.terminate()
print(response)

None
